# Reads

In [ ]:
Figura:
como alguem faria para usar o preditor
como alguem faria para testar com outra tecnologia o preditor
explicar que ele he independente de tecnologia
fluxograma mostrando a parte do ORDF e a parte do INNOVUS para geracao dos benchamrks
dps para geracao dos dados de treino e inferencia.
Explicar o problema dos dados, não é só uma questão de armazenamento, mas a leitura deles
tbm seria muito demorada inviabilizando o treino do modelo.

# Imports

In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sklearn
import tensorflow as tf
import random
import shutil
import os
import time
import re

# Learning Models

In [ ]:
evalMetrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc')]

# FCN Model
def makeFCNModel():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (22, 33, 33)))
  model.add(tf.keras.layers.MaxPooling2D((3, 3)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# CNN Model
def makeCNNModel():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (22, 33, 33)))
  model.add(tf.keras.layers.MaxPooling2D((3, 3)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation = 'relu'))#Dense
  model.add(tf.keras.layers.Dense(128, activation = 'relu'))#Dense
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# Select Training Data

In [ ]:
df = pd.read_pickle('benchmark.pkl', compression='zip')
# Get ONLY DRV CLEAN routed circuits
df = df.loc[df['FDRTotal'] == 0]
# Get all designs to train
circuitsToTrain = [x for x in df.loc[df['Design'].str.contains('80') == False].sort_values(by=['Design'])['Design']]

allPkls = ['/data/CSV/' + x[0:-3] + '/cts_' + x + '.pkl' for x in circuitsToTrain]
allPkls = [x for x in allPkls if os.path.exists(x)]
# Get only designs within 75 and 85 except 80
allPkls = [x for x in allPkls if int(re.findall(r'\d+', x)[0]) < 86 and int(re.findall(r'\d+', x)[0]) > 74]

# Traning

In [ ]:
pos = 3051
neg = 798472
total = pos+neg
w0 = total/(2*neg)
w1 = total/(2*pos)
# w1 /= 4
weights = {0: w0, 1: w1}
sizeBatch = 64  # almost 10% of chance to have viol \
                # is important to ensure that each batch has a decent chance of containing a few positive samples
# weights = {0: 0.5, 1: 100}

def train(pklsForTraining, learningModel, modelPath, epochStart, epochEnd, trainResultDF = pd.DataFrame()):
  pkls = pklsForTraining.copy()
  for epoch in range(epochStart, epochEnd):
    random.shuffle(pkls)
    for pkl in pkls:
      trainDf = pd.read_pickle(pkl, compression='zip')
      trainDf = trainDf.reset_index(drop=True)
      valDf = trainDf.sample(frac=0.2)
      trainDf = trainDf.drop(valDf.index)

      labels = trainDf.pop(trainDf.columns.values[-1])
      valLabels = valDf.pop(valDf.columns.values[-1])
      trainHyperImages = np.array(trainDf).reshape(len(trainDf),22,33,33)
      valHyperImages = np.array(valDf).reshape(len(valDf),22,33,33)
      print('Epoch: ',epoch,' Training with:', pkl)
      train_history = learningModel.fit(x=trainHyperImages,
                                       y=labels,
                                       verbose=2, #0 = silent, 1 = progress bar, 2 = one line per epoch
                                       batch_size=sizeBatch,
                                       validation_data=(valHyperImages, valLabels),
                                       class_weight=weights)
      historyDf = pd.DataFrame(train_history.history)
      historyDf['epoch'] = epoch
      historyDf['design'] = pkl[pkl.rfind('/')+5:pkl.find('.')]
      trainResultDF = pd.concat([trainResultDF, historyDf])
    pickle.dump(learningModel, open(modelPath+'model_'+str(epoch)+'.pkl', 'wb'))
    pickle.dump(trainResultDF, open(modelPath+'trainResultDF.pkl', 'wb'))

In [ ]:
modelPath = 'cnn/'
useFCN = False

In [ ]:
modelPath = 'fcn/'
useFCN = True

In [ ]:
numEpochs = 5

if os.path.exists(modelPath) == False:
  os.mkdir(modelPath)

models = [x for x in os.listdir(modelPath)]
lastRunEpoch = 0
learningModel = None
trainResultDF = pd.DataFrame()
if len(models) > 0:
  if 'trainResultDF.pkl' in models:
    models.remove('trainResultDF.pkl')
  models.sort()
  lastModel = models[-1]
  lastRunEpoch = int(lastModel[lastModel.find('_')+1:lastModel.find('.')])
  learningModel = pickle.load(open(modelPath+'model_'+str(lastRunEpoch)+'.pkl', 'rb'))
  lastRunEpoch += 1
  trainResultDF = pickle.load(open(modelPath+'trainResultDF.pkl', 'rb'))
else:
  learningModel = makeFCNModel() if useFCN else makeCNNModel()

train(allPkls, learningModel, modelPath, lastRunEpoch, numEpochs, trainResultDF)

# Cross Validation

In [ ]:
def predict(model, pkl):
  testDf = pd.read_pickle(pkl, compression='zip')
  labels = testDf.pop(testDf.columns.values[-1])
  testHyperImages = np.array(testDf).reshape(len(testDf),22,33,33)
  result = model.evaluate(testHyperImages, labels)
  resultDict = {m:r for (m, r) in zip(model.metrics_names, result)}
  return resultDict

#circuitsToTest = [x for x in os.listdir('/data/CSVWhole/')]
#random.shuffle(circuitsToTest)
circuitsToTest = ['tinyRocket', 'jpeg', 'swerv', 'bp_fe', 'bp_be', 'swerv_wrapper', 'dynamic_node', 'bp_multi', 'aes', 'ibex']
circuitPaths = ['/data/CSVWhole/'+x+'/' for x in circuitsToTest]

crossValItrs = []
i = 0
for x in range(len(circuitPaths)):
  if x % 2 == 0:
    crossValItrs.append(('Itr'+str(i), circuitPaths[x], circuitPaths[x+1]))
    i += 1

resultDF = pd.DataFrame()
for itrCV in crossValItrs:
  valPkls = [itrCV[1]+x for x in os.listdir(itrCV[1])]
  valPkls += [itrCV[2]+x for x in os.listdir(itrCV[2])]
  model = pickle.load(open(itrCV[0]+'/model_4.pkl', 'rb'))
  itrDf = pd.DataFrame()
  for pkl in valPkls:
    resultDict = predict(model, pkl)
    resultDf = pd.DataFrame(resultDict, index=[0])
    resultDf['Itr'] = itrCV[0]
    itrDf = pd.concat([itrDf, resultDf], ignore_index=True)
  itrDf = itrDf.sum(axis=0)
  itrDf = pd.DataFrame(itrDf).T
  resultDF = pd.concat([resultDF, itrDf], ignore_index=True)
pickle.dump(resultDF, open('finalDFCV.pkl', 'wb'))

In [ ]:
result = pickle.load(open('finalDFCV.pkl', 'rb'))
result

In [ ]:
rows = []
for index, row in result.iterrows():
  tp = row['tp']
  tn = row['tn']
  fp = row['fp']
  fn = row['fn']
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)
  TotalPos = tp+fn
  TotalNeg = fp+tn
  PosRatio = ((tp+fn)/(tp+fn+fp+tn))*100
  tpr = tp/(tp+fn)
  spc = tn/(tn+fp)
  fa = fp/(tn+fp)
  acc = (tp+tn)/(tp+tn+fp+fn)
  sqrt = math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
  mcc = 0
  if sqrt != 0:
    mcc = ((tp * tn) - (fp * fn))/sqrt
  rows.append({'tp':tp, 'tn':tn, 'fp':fp, 'fn':fn,
               'PosRatio%':PosRatio,
               'tpr':tpr, 'spc':spc, 'acc':acc, 'MCC[-1:1]':mcc})
  print()

valCircuits = []
i = 0
for x in range(len(circuitsToTest)):
  if x % 2 == 0:
    valCircuits.append((circuitsToTest[x]+', '+circuitsToTest[x+1]))
    i += 1
crossValItrs

crossValResultDF = pd.DataFrame(rows, index=valCircuits)
crossValResultDF.loc['mean'] = crossValResultDF.mean()
crossValResultDF.index.name = 'Circuits'
crossValResultDF

# Loss Evaluation

In [ ]:
# for model in ['cnn', 'fcn']:
for model in ['Itr0', 'Itr1', 'Itr2', 'Itr3', 'Itr4']:
  df = pickle.load(open(model+'/trainResultDF.pkl', 'rb'))
  for epoch in range(min(df['epoch']), max(df['epoch'])+1):
    tp = sum(df.loc[df['epoch'] == epoch]['tp'])
    tn = sum(df.loc[df['epoch'] == epoch]['tn'])
    fp = sum(df.loc[df['epoch'] == epoch]['fp'])
    fn = sum(df.loc[df['epoch'] == epoch]['fn'])
    loss = sum(df.loc[df['epoch'] == epoch]['loss'])
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    print('model:', model, 'epoch:', epoch, 'loss:', loss, 'tp:', tp, 'tn:', tn, 'fp:', fp, 'fn:', fn, 'precision:', precision, 'recall:', recall)
  print()

In [ ]:
dataPath = '/data/CSVWhole/'
modelsToEval = ['fcn/model_4.pkl', 'cnn/model_1.pkl']
finalDF = pd.DataFrame()
for model in modelsToEval:
  learningModel = pickle.load(open(model, 'rb'))
  for design in os.listdir(dataPath):
    pkls = [x for x in os.listdir(dataPath+design)]
    pkls.sort(key=lambda x: int(x[x.rfind('_')+1:x.rfind('.')]))
    for pkl in pkls:
      resultDict = predict(learningModel, dataPath+design+'/'+pkl)
      resultDf = pd.DataFrame(resultDict, index=[0])
      resultDf['file'] = int(pkl[pkl.rfind('_')+1:pkl.find('.')])
      resultDf['design'] = pkl[pkl.find('_')+1:pkl.rfind('_')]
      resultDf['model'] = model[:model.find('.')]
      finalDF = pd.concat([finalDF, resultDf], ignore_index=True)
    pickle.dump(finalDF, open('finalDF.pkl', 'wb'))

In [ ]:
model = pickle.load(open('exp9Models/model_4.pkl', 'rb'))

# Load Model

In [ ]:
trainResultDF = pickle.load(open('EXP9trainResultDF.pkl', 'rb'))
trainResultDF.shape
# plt.plot(train_history.history['loss'][0:50])
# plt.plot(train_history.history['val_loss'][0:50])

In [ ]:
for epoch in range(min(trainResultDF['epoch']), max(trainResultDF['epoch'])+1):
  loss = sum(trainResultDF.loc[trainResultDF['epoch'] == epoch]['loss'])
  valLoss = sum(trainResultDF.loc[trainResultDF['epoch'] == epoch]['val_loss'])
  print('epoch:', epoch, ' loss: ', loss, ' valLoss: ', valLoss)

In [ ]:
design = 'jpeg_79'
sortedDF = trainResultDF.loc[trainResultDF['design'] == design].sort_values(by=['epoch'])
ytrain = [x for x in sortedDF['loss']]
yval = [x for x in sortedDF['val_loss']]
plt.plot(ytrain, label = "ytrain")
plt.plot(yval, label = "yval")
plt.legend()
plt.title(design)
plt.show()

# Predict

In [ ]:
def predict(model, pkl):
  testDf = pd.read_pickle(pkl, compression='zip')
  labels = testDf.pop(testDf.columns.values[-1])
  testHyperImages = np.array(testDf).reshape(len(testDf),22,33,33)
  result = model.evaluate(testHyperImages, labels)
  resultDict = {m:r for (m, r) in zip(model.metrics_names, result)}
  return resultDict

In [ ]:
dataPath = '/data/CSVWhole/'
modelsToEval = ['fcn/model_4.pkl', 'cnn/model_1.pkl']
finalDF = pd.DataFrame()
for model in modelsToEval:
  learningModel = pickle.load(open(model, 'rb'))
  for design in os.listdir(dataPath):
    pkls = [x for x in os.listdir(dataPath+design)]
    pkls.sort(key=lambda x: int(x[x.rfind('_')+1:x.rfind('.')]))
    for pkl in pkls:
      resultDict = predict(learningModel, dataPath+design+'/'+pkl)
      resultDf = pd.DataFrame(resultDict, index=[0])
      resultDf['file'] = int(pkl[pkl.rfind('_')+1:pkl.find('.')])
      resultDf['design'] = pkl[pkl.find('_')+1:pkl.rfind('_')]
      resultDf['model'] = model[:model.find('.')]
      finalDF = pd.concat([finalDF, resultDf], ignore_index=True)
    pickle.dump(finalDF, open('finalDF.pkl', 'wb'))

In [ ]:
designs = set()
for x in finalDF['design']:
  designs.add(x)
designs
for x in designs:
  print(x, sum(finalDF.loc[finalDF['design'] == x]['tp']))

# Evaluate

In [ ]:
def calculate_test_metrics(model, results):
  m = {}
  for name, value in zip(model.metrics_names, results):
    m[name] = value
  if m['precision'] + m['recall'] != 0:
    f_score = 2 * ((m['precision'] * m['recall'])/(m['precision'] + m['recall']))
    m['F-score'] = f_score
  else:
    m['F-score'] = 0
  sqrt = math.sqrt((m['tp']+m['fp'])*(m['tp']+m['fn'])*(m['tn']+m['fp'])*(m['tn']+m['fn']))
  if sqrt != 0:
    mcc = ((m['tp'] * m['tn']) - (m['fp'] * m['fn']))/sqrt
    m['MCC'] = mcc
  else:
    m['MCC'] = -1
  return m

In [ ]:
outputPath = 'cnn/inference.pkl'
sizeBatch = 64 # is important to ensure that each batch has a decent chance of containing a few positive samples
for circuit in os.listdir('/data/CSVWhole/'):
  i = 0
  resultDF = pd.DataFrame()
  for pkl in ['/data/CSVWhole/'+circuit+'/'+x for x in os.listdir('/data/CSVWhole/'+circuit)]:
    print(circuit, i, pkl)
    testDF = pd.read_pickle(pkl, compression='zip')
    testDF = testDF.sample(frac=1).reset_index(drop=True) #Shuffle all rows
    testLabels = testDF.pop(testDF.columns.values[-1])
    testHyperImages = np.array(testDF).reshape(len(testDF),22,33,33)
    baseline_results = model.evaluate(x=testHyperImages,
                                      y=testLabels,
                                      batch_size=sizeBatch)
    test_metrics = calculate_test_metrics(model, baseline_results)
    testDF = pd.DataFrame(test_metrics, index=[i])
    testDF['design'] = circuit
    resultDF = pd.concat([resultDF, testDF], ignore_index=True)
    i += 1
pickle.dump(resultDF, open(outputPath, 'wb'))

# Benchmark Plot

In [ ]:
df = pd.read_pickle('benchmark.pkl', compression='zip')

In [ ]:
# Bar plot Routing Runtime Break Down
dfRuntime = df.loc[df['Design'].str.contains('80')].sort_values(by=['COMPONENTS'], ascending=False)
dfRuntime['Design'] = [x[0:x.rfind('_')] for x in dfRuntime['Design']]
dfRuntime = dfRuntime.set_index('Design')
dfRuntime = dfRuntime[['GR', 'IDR', 'FDR']]
ax = dfRuntime.plot.bar(stacked=True)
ax.set_ylabel('Runtime in seconds')
ax.set_xlabel('Designs (80% row utilization)')
ax.legend(["Global Routing", "Initial Detailed Routing", "Complete Detailed Routing"])
plt.title('Routing Runtime Break Down For 80% Row Utilization Designs')
# plt.savefig('routing_runtime.pdf')

In [ ]:
# TODO adicionar nas barras laranjas os numeros de IDRVs

In [ ]:
# Bar plot Routing Runtime Break Down
dfRuntime = df.loc[df['Design'].str.contains('80')].sort_values(by=['COMPONENTS'], ascending=False)
dfRuntime['Design'] = [x[0:x.rfind('_')] for x in dfRuntime['Design']]
dfRuntime = dfRuntime.set_index('Design')
dfRuntime = dfRuntime[['IDRShort']]
ax = dfRuntime.plot.bar(stacked=True)
ax.set_ylabel('Initial Detailed Routing Violations (IDRVs)')
ax.set_xlabel('Designs (80% row utilization)')
plt.title('Number of IDRVs For 80% Row Utilization Designs')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
idrShort = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  avgIDRShort = sum(tempDf['IDRShort']/len(tempDf))
  idrShort.append(avgIDRShort)
    
plt.plot([y for y in range(70, 91)], idrShort, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Initial Detailed Routing Short Violations (IDRV)")
plt.title('Average IDR Short x Row Utilization (Only DRV Free Designs)')

In [ ]:
# Adicionar o valor maximo do eixo y

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
fdrRuntime = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  fdr = sum(tempDf['FDR']/len(tempDf))
  fdrRuntime.append(fdr)

plt.plot([y for y in range(70, 91)], fdrRuntime, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Runtime (seconds)")
plt.title('Average Runtime to complete routing (Only DRV Free Designs)')

# Benchmark Info

# Compress All CSVs

# Backup

In [ ]:
# scaler = sklearn.preprocessing.StandardScaler()
# trainHyperImages = scaler.fit_transform(trainHyperImages)
# valHyperImages = scaler.transform(valHyperImages)

# scaler = sklearn.preprocessing.StandardScaler()
# labels = df[33*33*22]
# df[0:33*33*22-1] = scaler.fit_transform(df[0:33*33*22-1]).round(decimals=2)
# df[33*33*22] = labels

In [ ]:
# df = df.sample(frac=1).reset_index(drop=True) #Shuffle all rows
# dfVal = df.sample(frac=0.2)
# df = df.drop(dfVal.index)

# labels = df.pop(df.columns.values[-1])
# valLabels = dfVal.pop(dfVal.columns.values[-1])
# trainHyperImages = np.array(df).reshape(len(df),22,33,33)
# valHyperImages = np.array(dfVal).reshape(len(dfVal),22,33,33)